## Data processing

This Jupyter Noterbook helps us to convert binary attribute(s) to +/-1, categorical attributes(s) to onehot.

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

We load the data which were cleaned from the `data cleaning` step.

In [2]:
Xy = np.loadtxt('septic_cleaned.dat', dtype = 'str')

print(Xy.shape)
print(Xy)

(138, 48)
[['2.0' '10.4' '6.3' ... '1.0' '1.0' '1.0']
 ['2.0' '83.8' '27.1' ... '1.0' '1.0' '0.0']
 ['2.0' '63.3' '51.4' ... '1.0' '1.0' '0.0']
 ...
 ['2.0' '38.4' '21.477272727272723' ... '2.0' '1.0' '1.0']
 ['2.0' '68.1' '79.2' ... '1.0' '1.0' '1.0']
 ['2.0' '4.0' '1.5' ... '1.0' '1.0' '0.0']]


### Attributes

We find number of unique value for each column, to have an idea about which variables are continuous, which variables are binary, category. It depends on data, however it is likely that nu = 2 --> binary; nu = 3 or 4: --> category, n > 4: continuous. Of course, we have to see data in detail as well.

In [3]:
X = Xy[:,:-1]
l,n = X.shape
nu = np.array([len(np.unique(X[:,i])) for i in range(n)])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[  1 122 117 107  88  92  84  89  75  61  32   2   2  33  15  13  11  59
  30  52  66   3   4   3   1   4  51  76  62   2   2   8   2   7   1   1
   1   1 124   4   4   3   5   4   3   4   3]


In [4]:
i_binary = []
i_category = []
i_continuous = []
for i in range(X.shape[1]):
    nu = np.unique(X[:,i])
    if len(nu) == 2: # binary 
        i_binary.append(i)
    elif len(nu) < 5:
        i_category.append(i)
    else:
        i_continuous.append(i)
        
print('i_binary:',i_binary)
print('i_category:',i_category)

i_binary: [11, 12, 29, 30, 32]
i_category: [0, 21, 22, 23, 24, 25, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46]


We then define variable type, 1: continuous, 2: binary, 3: category.

In [5]:
variable_type  = np.ones(n) # continuous
variable_type[i_binary] = 2 # binary
variable_type[i_category] = 3 # categorical

print(variable_type)

[3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 3. 3. 3.
 3. 3. 1. 1. 1. 2. 2. 1. 2. 1. 3. 3. 3. 3. 1. 3. 3. 3. 1. 3. 3. 3. 3.]


We now convert binary to +/-1, category to onehot.

In [6]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """
    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # continuous
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))
        elif i_type == 2: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))
        else: # category      
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))        

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [7]:
# convert X
X_new = convert_binary_and_category(X,variable_type)

print(X_new.shape)
print(X_new)

(138, 75)
[[ 1.         10.4         6.3        ...  0.          0.
   1.        ]
 [ 1.         83.8        27.1        ...  0.          0.
   1.        ]
 [ 1.         63.3        51.4        ...  0.          0.
   1.        ]
 ...
 [ 1.         38.4        21.47727273 ...  0.          0.
   1.        ]
 [ 1.         68.1        79.2        ...  0.          0.
   1.        ]
 [ 1.          4.          1.5        ...  0.          0.
   1.        ]]


### Target

In [8]:
## target
y = Xy[:,-1].astype(float)

print(np.unique(y,return_counts=True))

(array([0., 1.]), array([102,  36]))


In [9]:
y_new = y
# convert taget to 0 and 1
#y_new = np.ones(y.shape[0])


print(np.unique(y_new,return_counts=True))

(array([0., 1.]), array([102,  36]))


In [10]:
# combine X and y
Xy_new = np.hstack((X_new,y_new[:,np.newaxis]))

In [11]:
np.savetxt('septic_processed.dat',Xy_new,fmt='%f')